### Discounting Amazon (AMZN) Cash Flows

In [ ]:
import edgar
import pandas as pd

edgar.get_docs('amzn', 16, ['10-Q', '10-K'])

In [183]:
AMZN_TAGS = {
    'amzn:fulfillmentexpense': 'Fulfillment',
    'amzn:technologyandinfrastructureexpense': 'Technology and Infrastructure',
    'amzn:technologyandcontentexpense': 'Technology and Content'
}

amzn_folders = ['2023Q3', '2023Q2', '2023Q1', '2022Q4', '2022Q3', '2022Q2', '2022Q1', '2021Q4', '2021Q3', '2021Q2', '2021Q1', '2020Q4', '2020Q3', '2020Q2', '2020Q1']
data = []
for folder in amzn_folders:
    folder_path = f"edgar_data/amzn/{folder}/"
    ops_context_ref = edgar.get_context_ref_by_name(f"{folder_path}MetaLinks.json", 'Consolidated Statements of Operations')
    print(folder, ops_context_ref)
    data.append(edgar.get_report_data_by_ref(f"{folder_path}ReportData.xml", ops_context_ref, AMZN_TAGS))

df = pd.DataFrame(data=data, columns=[
    'Revenue',
    'COGS',
    'Gross profit',
    'Fulfillment',
    'Technology and Infrastructure',
    'Technology and Content',
    'Sales and Marketing',
    'General Administrative',
    'Other Operating Expenses (Income) Net',
    'Operating Expenses',
    'Operating Income',
    'Interest Income',
    'Interest Expense',
    'Other Income (Expense) Net',
    'Total Non-Operating Income (Expense)',
    'Pretax Income',
    'Taxes',
    'Equity-method investment activity, net of tax',
    'Net income',
    'EPS',
    'Shares',
    'start_date',
    'end_date'
]).fillna(0)
df['Gross profit'] = df['Revenue']-df['COGS']
df.iloc[3,:-2] -= df.iloc[4,:-2]+df.iloc[5,:-2]+df.iloc[6,:-2]
df.iloc[7,:-2] -= df.iloc[8,:-2]+df.iloc[9,:-2]+df.iloc[10,:-2]
df.iloc[11,:-2] -= df.iloc[12,:-2]+df.iloc[13,:-2]+df.iloc[14,:-2]


2023Q3 c-10
2023Q2 c-10
2023Q1 id5a05ce5ce084612b9b02c88a7bcfc49_D20230101-20230331
2022Q4 i66a08f9a87424ace8d8c03c38fb30db9_D20220101-20221231
2022Q3 i6a7be016b1e14018ba8f205a6d0b0ff6_D20220701-20220930
2022Q2 if1c898010eef4246b9b88f9dfa8610f5_D20220401-20220630
2022Q1 iee090cd6862849ad8e3026c04d3a2165_D20220101-20220331
2021Q4 i1c47188d91bf419ea3e135b738fc8996_D20210101-20211231
2021Q3 id570d7b8c6b34b6aae878fab6e85facb_D20210701-20210930
2021Q2 i1d3899c8390c46559c77ab89c8be5417_D20210401-20210630
2021Q1 i1e039a3db3bf4fb1a3f80147e19ceb02_D20210101-20210331
2020Q4 iae13f4918ade40ed89c5d6d841ce2fd8_D20200101-20201231
2020Q3 ibd383e58f8cf40ef9dd044e6f3ad3549_D20200701-20200930
2020Q2 if7109a9e736a4d39bad498983cfb75d0_D20200401-20200630
2020Q1 FD2020Q1YTD


In [184]:
margins = pd.DataFrame({
    'Quarter': df['end_date'].dt.to_period('Q'),
    'Gross Margin': df['Gross profit']/df['Revenue'],
    'Operating Margin': df['Operating Income']/df['Revenue'],
    'Net Margin': df['Net income']/df['Revenue'],
    'Tax Rate': df['Taxes']/df['Pretax Income']
}).round(2)
print(f"avg gross margin: {margins['Gross Margin'].mean()}")
margins

avg gross margin: 0.4313333333333333


,Quarter,Gross Margin,Operating Margin,Net Margin,Tax Rate
0,2023Q3,0.48,0.08,0.07,0.19
1,2023Q2,0.48,0.06,0.05,0.11
2,2023Q1,0.47,0.04,0.02,0.23
3,2022Q4,0.43,0.02,0.00,1.28
4,2022Q3,0.45,0.02,0.02,0.02
5,2022Q2,0.45,0.03,-0.02,0.24
6,2022Q1,0.43,0.03,-0.03,0.27
7,2021Q4,0.40,0.03,0.10,0.04
8,2021Q3,0.43,0.04,0.03,0.27
9,2021Q2,0.43,0.07,0.07,0.10


In [244]:
forecast_years = 20
growth_rate = 0.12
cost_growth_rate = 0.05
expected_gross_margin = 0.43
tax_rate = 0.2

forecast_data = [df.iloc[0].copy()]
for year in range(1, forecast_years+1):
    forecast = df.iloc[0].copy()
    forecast['start_date'] = forecast_data[year-1]['start_date']+pd.offsets.DateOffset(years=1)
    forecast['end_date'] = forecast_data[year-1]['end_date']+pd.offsets.DateOffset(years=1)
    forecast['Revenue'] = forecast_data[year-1]['Revenue']*(1+growth_rate)
    forecast['Gross profit'] = forecast['Revenue']*expected_gross_margin
    forecast['COGS'] = forecast['Revenue']-forecast['Gross profit']
    forecast['Operating Expenses'] = forecast['COGS']
    for k in ['Fulfillment', 'Technology and Infrastructure', 'Technology and Content', 'Sales and Marketing', 'General Administrative', 'Other Operating Expenses (Income) Net']:
        forecast[k] = forecast_data[year-1][k]*(1+cost_growth_rate)
        forecast['Operating Expenses'] += forecast[k]
    forecast['Operating Income'] = forecast['Revenue']-forecast['Operating Expenses']
    forecast['Interest Income'] = forecast_data[year-1]['Interest Income']*(1+growth_rate)
    forecast['Interest Expense'] = forecast_data[year-1]['Interest Expense']*(1+growth_rate)
    forecast['Other Income (Expense) Net'] = forecast_data[year-1]['Other Income (Expense) Net']*(1+growth_rate)
    forecast['Total Non-Operating Income (Expense)'] = forecast['Operating Income']+forecast['Interest Income']-forecast['Interest Expense']-forecast['Other Income (Expense) Net']
    forecast['Pretax Income'] = forecast['Revenue']-forecast['Operating Expenses']
    forecast['Taxes'] = forecast['Pretax Income']*tax_rate
    forecast['Net income'] = forecast['Pretax Income']-forecast['Taxes']+forecast['Equity-method investment activity, net of tax']
    forecast['EPS'] = forecast['Net income']/forecast['Shares']

    forecast_data.append(forecast)

forecast = pd.DataFrame(data=forecast_data[1:]).reset_index(drop=True).round(2)
forecast

,Revenue,COGS,Gross profit,Fulfillment,Technology and Infrastructure,Technology and Content,Sales and Marketing,General Administrative,Other Operating Expenses (Income) Net,Operating Expenses,Operating Income,Interest Income,Interest Expense,Other Income (Expense) Net,Total Non-Operating Income (Expense),Pretax Income,Taxes,"Equity-method investment activity, net of tax",Net income,EPS,Shares,start_date,end_date
0,160252.96,91344.19,68908.77,23429.70,22263.15,0.0,11078.55,2689.05,-256.20,150548.44,9704.52,869.12,902.72,1154.72,8516.20,9704.52,1940.90,-4,7759.62,0.73,10558,2024-07-01,2024-09-30
1,179483.32,102305.49,77177.83,24601.18,23376.31,0.0,11632.48,2823.50,-269.01,164469.95,15013.36,973.41,1011.05,1293.29,13682.44,15013.36,3002.67,-4,12006.69,1.14,10558,2025-07-01,2025-09-30
2,201021.31,114582.15,86439.16,25831.24,24545.12,0.0,12214.10,2964.68,-282.46,179854.83,21166.48,1090.22,1132.37,1448.48,19675.85,21166.48,4233.30,-4,16929.18,1.60,10558,2026-07-01,2026-09-30
3,225143.87,128332.01,96811.86,27122.81,25772.38,0.0,12824.81,3112.91,-296.58,196868.33,28275.54,1221.05,1268.26,1622.30,26606.04,28275.54,5655.11,-4,22616.44,2.14,10558,2027-07-01,2027-09-30
4,252161.14,143731.85,108429.29,28478.95,27061.00,0.0,13466.05,3268.56,-311.41,215694.98,36466.15,1367.58,1420.45,1816.97,34596.31,36466.15,7293.23,-4,29168.92,2.76,10558,2028-07-01,2028-09-30
5,282420.47,160979.67,121440.80,29902.89,28414.05,0.0,14139.35,3431.98,-326.98,236540.96,45879.51,1531.69,1590.90,2035.01,43785.28,45879.51,9175.90,-4,36699.61,3.48,10558,2029-07-01,2029-09-30
6,316310.93,180297.23,136013.70,31398.04,29834.75,0.0,14846.32,3603.58,-343.33,259636.59,56674.34,1715.49,1781.81,2279.21,54328.81,56674.34,11334.87,-4,45335.47,4.29,10558,2030-07-01,2030-09-30
7,354268.24,201932.90,152335.34,32967.94,31326.49,0.0,15588.63,3783.76,-360.50,285239.22,69029.02,1921.35,1995.63,2552.72,66402.02,69029.02,13805.80,-4,55219.21,5.23,10558,2031-07-01,2031-09-30
8,396780.43,226164.84,170615.58,34616.34,32892.81,0.0,16368.06,3972.95,-378.52,313636.49,83143.94,2151.91,2235.10,2859.04,80201.71,83143.94,16628.79,-4,66511.15,6.30,10558,2032-07-01,2032-09-30
9,444394.08,253304.63,191089.45,36347.15,34537.45,0.0,17186.47,4171.60,-397.45,345149.85,99244.23,2410.14,2503.31,3202.13,95948.93,99244.23,19848.85,-4,79391.38,7.52,10558,2033-07-01,2033-09-30


In [245]:
import importlib
importlib.reload(edgar)
npv = round(edgar.npv(forecast['Net income'], 0.0435), 2)
net_cash = 64169
total_value = npv+net_cash
{
    'npv': npv,
    'net_cash': net_cash,
    'total_value': total_value,
    'value_per_share': round(total_value/(df.iloc[0].loc['Shares']), 2)
}

{'npv': 1282068.3,
 'net_cash': 64169,
 'total_value': 1346237.3,
 'value_per_share': 127.51}